<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2024/AdvML2024_ex13notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex13notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




板書や口頭で補足する前提なので，この notebook だけでは説明が不完全です．


In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# scikit-learn のいろいろ
from sklearn.datasets import make_moons, fetch_openml
from sklearn.model_selection import train_test_split

# NumPy の 疑似乱数生成器（rng = random number generator）
from numpy.random import default_rng
rng = default_rng() # 疑似乱数生成器を初期化

---
## 主成分分析による次元削減（前回の続き）
---


［主成分分析の問題の解］（前回の復習）

$N$個の$D$次元ベクトルから成る学習データが与えられるとする．
$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$
ただし，このデータの平均は $\mathbf{0}$ とする．
また，$\mathbf{x}$ の分散共分散行列 $V = \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n\mathbf{x}_n^{\top}$ の固有値を $\lambda_1 \geq \lambda_2 \geq \ldots \geq \lambda_D$とし，これらに対応する単位固有ベクトルをそれぞれ $\mathbf{u}_1, \mathbf{u}_2,\ldots,\mathbf{u}_D$ とおく．

$H\times D$行列 $W$ によって
$$
\mathbf{z} = W\mathbf{x} = \begin{pmatrix}
\mathbf{w}_1\cdot\mathbf{x}\\
\mathbf{w}_2\cdot\mathbf{x}\\
\vdots\\
\mathbf{w}_H\cdot\mathbf{x}
\end{pmatrix}
$$
と次元削減するとき，主成分分析の目的を実現するためには，
ベクトル $\mathbf{w}_h$ ($h = 1, 2, \ldots, H$) を $\mathbf{u}_h$ の向きにとればよい（注）．

ここで，$\lambda_1$ から $\lambda_H$ （$H \leq D$）に対応する固有ベクトル $\mathbf{u}_1, \mathbf{u}_2, \ldots, \mathbf{u}_H$ をならべた，次のような行列 $U_H$ を考える．

$$
U_H = (\mathbf{u}_1\  \mathbf{u}_2\  \ldots\ \mathbf{u}_H )
$$

$U_H$ は $D \times H$ 行列である．このとき，主成分分析の変換行列 $W$ は，

$$
W = U_H^{\top} = \begin{pmatrix} \mathbf{u}_1^{\top} \\ \mathbf{u}_2^{\top} \\ \vdots \\ \mathbf{u}_H^{\top} \end{pmatrix}
$$

となる．

※ 注: ここで考えている固有ベクトルは「単位」固有ベクトル（大きさが1，すなわち $||\mathbf{u}_h||=1$）に限定しているが，符号の不定性がある（$\mathbf{u}_h$も$−\mathbf{u}_h$も固有ベクトル）．したがって，$\mathbf{w}_h = -\mathbf{u}_h$ としてもよい．


$\mathbf{x}_n$ の平均が $\mathbf{0}$ でない場合に一般化すると，

$$
\mathbf{z} = U_H^{\top}(\mathbf{x} - \mathbf{\mu}) = \begin{pmatrix}
\mathbf{u}_1\cdot(\mathbf{x} - \mathbf{\mu})\\
\mathbf{u}_2\cdot(\mathbf{x} - \mathbf{\mu})\\
\vdots\\
\mathbf{u}_H\cdot(\mathbf{x} - \mathbf{\mu})
\end{pmatrix}
$$

### 変換後のデータの平均と分散

学習データ $\mathbf{x}_n$ の平均が $\mathbf{\mu}$ のとき，変換後のデータ $\mathbf{z}_n$ の平均を $\mathbf{\mu}_z$ とおくと，

$$
\begin{aligned}
\mathbf{\mu}_z &= \frac{1}{N}\sum_{n=1}^{N}\mathbf{z}_n = \frac{1}{N}\sum_{n=1}^{N} U_H^{\top}(\mathbf{x}_n - \mathbf{\mu}) = U_H^{\top}\left( \frac{1}{N}\sum_{n=1}^{N} (\mathbf{x}_n - \mathbf{\mu})\right)\\
&= U_H^{\top}\left( \frac{1}{N}\sum_{n=1}^{N}\mathbf{x}_n - \frac{1}{N}\sum_{n=1}^{N}\mathbf{\mu} \right) = U_H^{\top} \left( \mathbf{\mu} - \mathbf{\mu}\right) = \mathbf{0}
\end{aligned}
$$

となる．

学習データの分散共分散行列を $V$ とし，変換後のデータの分散共分散行列を $V_z$ とおくと，

$$
\begin{aligned}
V_z &= \frac{1}{N}\sum_{n=1}^{N}(\mathbf{z}_n - \mathbf{\mu}_z)(\mathbf{z}_n - \mathbf{\mu}_z)^{\top} = \frac{1}{N}\sum_{n=1}^{N}\mathbf{z}_n\mathbf{z}_n^{\top}\\
&= \frac{1}{N}\sum_{n=1}^{N} U_H^{\top}(\mathbf{x}_n - \mathbf{\mu}) \left(U_H^{\top}(\mathbf{x}_n - \mathbf{\mu})\right)^{\top}
= \frac{1}{N}\sum_{n=1}^{N} U_H^{\top}(\mathbf{x}_n - \mathbf{\mu}) (\mathbf{x}_n - \mathbf{\mu})^{\top} U_H\\
&= U_H^{\top} V U_H  = \textrm{diag}(\lambda_1, \lambda_2, \ldots, \lambda_H)
\end{aligned}
$$

となる．したがって，変換後のデータの変数間の共分散は（相関も） $0$ である．
また，変換後のデータの $h$ 番目の変数 $y_h$ の分散は，$V$ の $h$ 番目の固有値 $\lambda_h$ に等しい．

### 再構成とその誤差

$D$ 次元のデータ $\mathbf{x}$ に対して $\mathbf{z} = U_H^{\top}(\mathbf{x} - \mathbf{\mu})$ は $H$ 次元である．このとき，

$$
\widehat{\mathbf{x}} = U_H\mathbf{z} + \mathbf{\mu} = U_HU_H^{\top}(\mathbf{x} -  \mathbf{\mu}) + \mathbf{\mu}
$$

は $D$ 次元である．これは，$\mathbf{x}$ を $H$ 次元に次元削減してから元の次元数に戻したものになっており，「再構成」(reconstruction)と呼ばれる．

主成分分析は，学習データとその再構成との間の二乗誤差

$$
\sum_{n=1}^{N} \Vert \mathbf{x}_n - \widehat{\mathbf{x}}_n  \Vert^2
$$

を最小化する線形変換となっている．つまり，任意の $H\times D$ 行列 $A$ と $D \times H$ 行列 $B$ を用いて $\widehat{\mathbf{x}} = BA(\mathbf{x} - \mathbf{\mu}) + \mathbf{\mu}$ と再構成することを考えたとき，$A = U_H^{\top}, B = U_H$ とすると再構成の二乗誤差が最小となる（注）．

※ 注: 「最小になるのは $A = U_H^{\top}, B = U_H$ のときに限られる」とは言ってないことに注意．実際，任意の $H\times H$ 正則行列 $C$ に対して $A = CU_H^{\top}, B = U_H C^{-1}$ とおけば上記の二乗誤差は最小となる．

### PCAによる生成

主成分分析による次元削減と再構成の性質から，次のようなデータ生成モデル（注）を考えることができる．

1. $H$ 次元の潜在変数 $\mathbf{z}$ が次のような正規分布から生成される：
$$
{\cal N}(\mathbf{z}; \mathbf{0}, \Sigma_H)
$$
ただし， $\Sigma_H$ は $\Sigma_H = \textrm{diag}(\lambda_1^2, \lambda_2^2, \ldots, \lambda_H^2)$ という対角行列．
1. $\mathbf{z}$ に直交変換 $U_H$ を作用させて $D$ 次元にし，$\mathbf{\mu}$ だけ並行移動したものが $\mathbf{x}$ として観測される． $\mathbf{x} = U_H\mathbf{z} + \mathbf{\mu}$．

※ 注: この説明には確率モデルとして正確・厳密でないところがあります．主成分分析をきちんとした確率モデルとして定式化したものに，「確率的主成分分析」（Probabilistic PCA, PPCA）があります．

### 実験: 手書き数字画像の再構成

手書数字画像のデータセット MNIST のサブセットを用いて，PCAによる再構成と生成の実験をやってみよう．

#### データの準備

In [ ]:
# MNIST データセットの入手
Xraw, yraw = fetch_openml('mnist_784', version=1, parser='auto', return_X_y=True, as_frame=False)
Xall = Xraw[:20000] / 255.0     # 画素値が [0, 255] の整数値なので [0, 1] の浮動小数点数値に変換
yall = yraw[:20000].astype(int) # クラスラベル．0 から 9 の整数値

# 学習データとテストデータの分割
XL, XT, yL, yT = train_test_split(Xall, yall, test_size=4000, random_state=4649, stratify=yall)
print(XL.shape, yL.shape)
print(XT.shape, yT.shape)
NL, NT = len(XL), len(XT)

D = XL.shape[1]
#K = 10

In [ ]:
# 学習データの最初の50枚を可視化
nrow, ncol = 5, 10
fig, ax = plt.subplots(nrow, ncol, figsize=(0.8*ncol, 0.8*nrow))
for i in range(nrow):
    for j in range(ncol):
        img = XL[i*ncol + j, ::].reshape((28, 28))
        ax[i, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
        ax[i, j].axis('off')

fig.tight_layout()
plt.show()

#### $V$ の固有値と固有ベクトルの計算

ここでは，分散共分散行列を求める → その固有値固有ベクトルを求める，という計算をする代わりに，「特異値分解」(Singular Value Decomposition, SVD)という行列分解を経由して分散共分散行列の固有値固有ベクトルを求めています．
その方が良いことがいくつかあるからなのですが，説明は省略します．

In [ ]:
# 平均を引いたデータ行列を求める
Xm = np.mean(XL, axis=0)
XL2 = XL - Xm

# それをSVD
_, sva, Vt = np.linalg.svd(XL2, full_matrices=False)
eva = sva**2/len(XL2)  # 固有値
U = Vt                  # 固有ベクトルをならべた行列

# 固有値の大きい方 10 個の値
print(eva[:10])

In [ ]:
cumeva = np.cumsum(eva)
cumeva /= cumeva[-1]

# グラフ
fig = plt.figure(figsize=(10, 6))
# 固有値
ax1 = fig.add_subplot(221)
ax1.plot(np.arange(len(eva))+1, eva, label='eigenvalues')
ax1.axhline(0, color='gray')
ax1.legend()
# 固有値（次元数の範囲を限定）
ax2 = fig.add_subplot(222)
ax2.plot(np.arange(len(eva))+1, eva, '.', label='eigenvalues')
ax2.axhline(0, color='gray')
ax2.set_xlim(-5, 100)
ax2.legend()
# 累積寄与率
ax3 = fig.add_subplot(223)
ax3.plot(np.arange(len(eva))+1, cumeva, label='cumulative contribution rate')
ax3.axhline(1, color='gray')
ax3.axhline(0.9, color='gray', linestyle='--')
ax3.axhline(0.8, color='gray', linestyle='--')
ax3.legend()
# 累積寄与率（次元数の範囲を限定）
ax4 = fig.add_subplot(224)
ax4.plot(np.arange(len(eva))+1, cumeva, '.', label='cumulative contribution rate')
ax4.axhline(1, color='gray')
ax4.axhline(0.9, color='gray', linestyle='--')
ax4.axhline(0.8, color='gray', linestyle='--')
ax4.set_xlim(-5, 100)
ax4.legend()
fig.tight_layout()
plt.show()

上段はデータの分散共分散行列の固有値，下段はそこから算出される累積寄与率を示す．
いずれの図においても，横軸は次元数を表す．

固有値を降順に $\lambda_h$ ($h = 1, 2, \ldots, D)$ とおくとき，累積寄与率 $c_h$ は次式で計算される量である．

$$
c_h = \frac{\sum_{d=1}^{h}\lambda_d}{\sum_{d=1}^{D}\lambda_d}
$$

分母は $y_1$ から $y_D$ までの変数の分散の総和である．これは，元のデータの$D$個すべての変数の分散の総和に等しい．分子は，$y_1$ から $y_h$ までの変数の分散の総和である．したがって，$c_h$ は，変換後の $h$ 個の変数だけで元データの分散のうちどの程度の割合を表せるかを示す．

In [ ]:
# 平均と固有ベクトルを可視化
nrow, ncol = 5, 10
fig, ax = plt.subplots(nrow, ncol, figsize=(0.8*ncol, 0.8*nrow))
for i in range(nrow):
    for j in range(ncol):
        if i == 0 and j == 0:
            uu = Xm
        else:
            uu = U[i*ncol +j - 1, ::]
            uumax = np.max(np.abs(uu))
            uu = uu / (uumax*2) + 0.5
        img = uu.reshape((28, 28))
        ax[i, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
        ax[i, j].axis('off')

fig.tight_layout()
plt.show()

左上は平均を可視化したもの，それ以外は，固有ベクトルを可視化したものである．左上から右に向かって，対応する固有値の大きい順に並んでいる．固有ベクトルは正負の値から成るので，ここでは値 $0$ がちょうど真っ黒と真っ白の中間になるように規格化している．

#### 再構成

In [ ]:
# 次元数
H = 10

# テストデータを H 次元に次元削減
Z = (XT - Xm) @ U[:H, :].T

# 再構成
XX = Z @ U[:H, :] + Xm

# 再構成したテストデータの最初の10枚を可視化
ncol = 10
fig, ax = plt.subplots(2, ncol, figsize=(0.8*ncol, 0.8*2))

# 元画像
for j in range(ncol):
    img = XT[j, ::].reshape((28, 28))
    ax[0, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    ax[0, j].axis('off')

# 再構成した画像
for j in range(ncol):
    img = XX[j, ::].reshape((28, 28))
    ax[1, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    ax[1, j].axis('off')

fig.tight_layout()
plt.show()

上段のデータを $H$ 次元にしてから $D = 784$ 次元で再構成したものが下段．

In [ ]:
### 次元数を変えながら学習データを再構成して二乗誤差を計算

# 学習データを変換
Z = XL2 @ U.T

# 次元数 H の設定
HList = np.hstack((
    [0], np.arange(1, 20), np.arange(20, 50, 5),
    np.arange(50, 400, 10), np.arange(400, 700, 50), [700, D]
))

# 平均二乗誤差の配列
mse = np.zeros(len(HList))

# 0 番目は平均をそのまま再構成とした場合の二乗誤差
mse[0] = np.mean(XL2**2)

for i, H in enumerate(HList[1:]):
    # H次元で再構成
    XX = Z[:, :H] @ U[:H, :] + Xm
    # 元画像と再構成との間の平均二乗誤差
    mse[i] = np.mean((XL - XX)**2)
    print(f'{H} {mse[i]:.6f}')

# 二乗誤差をグラフに描く
fig = plt.figure(figsize=(9, 6))
ax1 = fig.add_subplot(111)
ax1.plot(HList, mse, '.', label='reconstruction error')
ax1.axhline(0, color='gray')
ax1.legend()
plt.show()

図は，横軸が次元数 $H$，縦軸が学習データとその再構成との間の二乗誤差．

#### 生成

In [ ]:
# 次元数
H = 10

# 生成
mu = np.zeros(H)
cov = np.diag(eva[:H])
Z = rng.multivariate_normal(np.zeros(H), cov, 50)
XX = Z @ U[:H, :] + Xm

# 生成した画像50枚を可視化
nrow, ncol = 5, 10
fig, ax = plt.subplots(nrow, ncol, figsize=(0.8*ncol, 0.8*nrow))
for i in range(nrow):
    for j in range(ncol):
        img = XX[i*ncol + j, ::].reshape((28, 28))
        ax[i, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
        ax[i, j].axis('off')

fig.tight_layout()
plt.show()

`H` をいろいろ変えて観察しよう．
潜在変数の分布が単一の正規分布であると仮定している，その線形変換でデータが生成されると仮定している，ということなので，あまり良い生成にはならない．

---
## 線形オートエンコーダ
---

---
### オートエンコーダとは

板書もしながら説明します．

- Encoder（符号化器）: $D$次元の値 $\mathbf{x}$ を入力すると $H < D$ 次元の値 $\mathbf{z}$ を出力する変換．$\mathbf{z} = \textrm{Enc}(\mathbf{x})$
- Decoder（復号器）: $H < D$ 次元の値 $\mathbf{z}$ を入力すると $D$ 次元の値 $\widehat{\mathbf{x}}$ を出力する変換．$\widehat{\mathbf{x}} = \textrm{Dec}(\mathbf{z})$

Enc, Dec をニューラルネットで構成し，$\widehat{\mathbf{x}} = \textrm{Dec}(\textrm{Enc}(\mathbf{x}))$ と $\mathbf{x}$ との間の誤差を最小化するように学習させると，$D$ 次元のデータを $H$ 次元にしてから $D$ 次元で再構成する仕組みを作れる．これを **オートエンコーダ** (**Autoencoder**) という．

学習データ

$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$

に対して，再構成との間の二乗誤差を損失関数とするのが一般的．

$$
L = \sum_{n=1}^{N} \Vert \mathbf{x}_n - \textrm{Dec}(\textrm{Enc}(\mathbf{x}_n))\Vert^2
$$



---
### 線形オートエンコーダ

オートエンコーダをニューラルネットで作る最も単純な方法は，$\textrm{Dec}(\textrm{Enc}(\mathbf{x}))$ を，中間層を一つだけもつ 2 層の階層型ニューラルネットワークで構成する方法である．

この場合，
中間層のニューロン数を $H < D$，出力層のニューロン数を $D$ として，上記の $L$ を損失関数として学習を行う．
出力層ニューロンの活性化関数は恒等関数とする（$\sigma(x) = x$とする）ことが多い（注）．

※ 注: 値の範囲が限られるデータを扱う場合，再構成される値が範囲からはみ出さないように，入力を $[0, 1]$ や $[-1, 1]$ に規格化したうえで，出力層ニューロンの活性化関数にロジスティックシグモイドや $\tanh$ を用いることもある．

このような2層ニューラルネットで $L$ を損失関数とする場合，学習によって得られる $L$ の値は，主成分分析で次元数を $H$ として次元削減→再構成した場合に得られる二乗誤差を下回らないことが知られている．
このことは，中間層のニューロンに非線形の活性化関数を用いても変わらない．

中間層ニューロンがバイアス項を持たず活性化関数が恒等関数である場合，この層には $H \times D$ 個のパラメータがある．それらの値を並べた $H\times D$ 行列を $A$ とおくと， $\textrm{Enc}(\mathbf{x}) = A\mathbf{x}$ と表せる．
また，出力層ニューロンもバイアス項を持たず活性化関数が恒等関数である場合，この層の $D \times H$ 個のパラメータを並べた $D \times H$ 行列を $B$ とおくと，$\textrm{Dec}(\mathbf{z}) = B\mathbf{z}$ と表せる．
中間層と出力層がこのように線形変換である場合，オートエンコーダ全体の入出力は $\textrm{Dec}(\textrm{Enc}(\mathbf{x})) = BA\mathbf{x}$ となる．
以下，このようなオートエンコーダを線形オートエンコーダと呼ぶことにする．

主成分分析の「再構成とその誤差」の節で述べたように，データの分散共分散行列の固有値の大きい方から $H$ 個に対応する固有ベクトルをならべた $D\times H$ 行列を $U_H$ としたとき， 任意の $H\times H$ 正則行列 $C$ に対して $A = CU_H^{\top}, B = U_H C^{-1}$ とおけば，再構成の二乗誤差が最小となる．
線形オートエンコーダのパラメータ $A, B$ は，学習によってこの形に収束することが知られている．



---
### 実験: 線形オートエンコーダによる手書き数字画像の次元削減と再構成

In [ ]:
# PyTorch 関係のほげ
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torchsummary

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# 平均を差し引いたデータ行列
Xm = np.mean(XL, axis=0)
XL2 = XL - Xm
XT2 = XT - Xm

In [ ]:
# データを扱うためのクラス
#
class MMDataset(Dataset):

    def __init__(self, dataX):
        self.X = dataX

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32)
        return X

In [ ]:
# 線形オートエンコーダ
#
class LinearAE(nn.Module):

    def __init__(self, D, H):
        super(LinearAE, self).__init__()
        # Encoder
        self.enc = nn.Linear(D, H, bias=False)
        # Decoder
        self.dec = nn.Linear(H, D, bias=False)

    def forward(self, X):
        X = self.enc(X)
        X = self.dec(X)
        return X

In [ ]:
# 学習の関数
#
def train(model, lossFunc, optimizer, dl):
    loss_sum = 0.0
    n = 0
    for i, X in enumerate(dl):
        X = X.to(device)
        Z = model(X)         # 一つのバッチ X を入力して出力 Z を計算
        loss = lossFunc(Z, X) # 入力 X を正解として loss を計算
        optimizer.zero_grad()   # 勾配をリセット
        loss.backward()         # 誤差逆伝播でパラメータ更新量を計算
        optimizer.step()         # パラメータを更新
        n += len(X)
        loss_sum += loss.item()  # 損失関数の値

    return loss_sum/n

In [ ]:
# 損失関数の値を求める関数
#
@torch.no_grad()
def evaluate(model, lossFunc, dl):
    loss_sum = 0.0
    n = 0
    for i, X in enumerate(dl):
        X = X.to(device)
        Z = model(X)           # 一つのバッチ X を入力して出力 Y を計算
        loss = lossFunc(Z, X)  # 入力 X を正解として loss を計算
        n += len(X)
        loss_sum += loss.item() # 損失関数の値

    return loss_sum/n

In [ ]:
# データ読み込みの仕組みを作る
dsL = MMDataset(XL - Xm)
dsT = MMDataset(XT - Xm)
dlL = DataLoader(dsL, batch_size=100, shuffle=True)
dlT = DataLoader(dsT, batch_size=100, shuffle=False)

In [ ]:
# ネットワークモデルの定義
H = 10
net = LinearAE(D, H).to(device)

# ネットワークの構造を表示
torchsummary.summary(net, (1, D))

# 損失関数（二乗誤差） とパラメータ最適化器の設定
loss_func = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

In [ ]:
# 学習の繰り返し回数
nepoch = 30

# 学習
L = []
print(f'学習データ数: {len(dsL)}  テストデータ数: {len(dsT)}')
print()
print('# epoch  lossL  lossT')
for t in range(1, nepoch+1):
    lossL = train(net, loss_func, optimizer, dlL) / D
    lossT = evaluate(net, loss_func, dlT) / D
    L.append([t, lossL, lossT])
    if (t < 10) or (t % 10 == 0):
        print(f'{t}   {lossL:.5f}   {lossT:.5f}')

# 学習曲線の表示
data = np.array(L)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(data[:, 0], data[:, 1], '.-', label='loss for training data')
ax.plot(data[:, 0], data[:, 2], '.-', label='loss for test data')
ax.axhline(0.0, color='gray')
ax.legend()
ax.set_title(f'loss')
plt.show()

# 学習後の損失
loss2 = evaluate(net, loss_func, dlL) / D
print(f'# lossL: {loss2:.5f}', end='   ')
loss2 = evaluate(net, loss_func, dlT) / D
print(f'# lossT: {loss2:.5f}')

In [ ]:
for i, X in enumerate(dlT):
    X = X.to(device)
    Z = net(X)
    break

XX = X.to('cpu').detach().numpy() + Xm
ZZ = Z.to('cpu').detach().numpy() + Xm


In [ ]:
# 最初の 1 バッチの出力を求める
for i, X in enumerate(dlT):
    X = X.to(device)
    Z = net(X)
    break

XX = X.to('cpu').detach().numpy() + Xm
ZZ = Z.to('cpu').detach().numpy() + Xm

# 再構成したテストデータの最初の10枚を可視化
ncol = 10
fig, ax = plt.subplots(2, ncol, figsize=(0.8*ncol, 0.8*2))

# 元画像
for j in range(ncol):
    img = XX[j, ::].reshape((28, 28))
    ax[0, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    ax[0, j].axis('off')

# 再構成した画像
for j in range(ncol):
    img = ZZ[j, ::].reshape((28, 28))
    ax[1, j].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)
    ax[1, j].axis('off')

fig.tight_layout()
plt.show()